In [1]:
from functools import partial

from transformers import CLIPVisionModel 
import torch
from torch import nn
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
from transformers import CLIPVisionModel
from torchvision import transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [2]:
train_pixel_img_feature = torch.load('/DATA/deep_learning/eeg-to-img/data/weights/EEG_Image_decode/ViT-L-14_features_GIT_train.pt')['img_features']# 
test_pixel_img_feature = torch.load('/DATA/deep_learning/eeg-to-img/data/weights/EEG_Image_decode/ViT-L-14_features_GIT_test.pt')['img_features']# 
train_img_feature = torch.load('/DATA/deep_learning/eeg-to-img/data/weights/EEG_Image_decode/ViT-H-14_features_train.pt')['img_features'].unsqueeze(1)# 
test_img_feature = torch.load('/DATA/deep_learning/eeg-to-img/data/weights/EEG_Image_decode/ViT-H-14_features_test.pt')['img_features'].unsqueeze(1)# 


In [3]:
print(test_img_feature.shape)
print(train_pixel_img_feature.shape)
print(train_img_feature.shape)
print(test_pixel_img_feature.shape)

torch.Size([200, 1, 1024])
torch.Size([16540, 257, 1024])
torch.Size([16540, 1, 1024])
torch.Size([200, 257, 1024])


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 257),
            nn.LayerNorm(257),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
        
        

# Instantiate the model, loss function, and optimizer

model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Prepare data loaders
train_dataset = TensorDataset(train_img_feature, train_pixel_img_feature)
test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/DATA/deep_learning/eeg-to-img/data/weights/EEG_Image_decode/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")


Epoch 1/30, Loss: 1.1396484375
Epoch 2/30, Loss: 0.7748531371124031
Epoch 3/30, Loss: 0.7653978924418605
Epoch 4/30, Loss: 0.7575778221899225
Epoch 5/30, Loss: 0.7516200339147286
Epoch 6/30, Loss: 0.7477819161821705
Epoch 7/30, Loss: 0.745139898255814
Epoch 8/30, Loss: 0.7420512354651163
Epoch 9/30, Loss: 0.7393637960271318
Epoch 10/30, Loss: 0.737789183624031
Epoch 11/30, Loss: 0.7351925872093024
Epoch 12/30, Loss: 0.7336709665697675
Epoch 13/30, Loss: 0.7321569161821705
Epoch 14/30, Loss: 0.7311197916666666
Epoch 15/30, Loss: 0.7306201550387597
Epoch 16/30, Loss: 0.7296208817829457
Epoch 17/30, Loss: 0.7290833938953488
Epoch 18/30, Loss: 0.7277434593023255
Epoch 19/30, Loss: 0.7273119549418605
Epoch 20/30, Loss: 0.7267820373062015
Epoch 21/30, Loss: 0.7260023013565892
Epoch 22/30, Loss: 0.7260098716085271
Epoch 23/30, Loss: 0.725563226744186
Epoch 24/30, Loss: 0.7243671269379846
Epoch 25/30, Loss: 0.7246396560077519
Epoch 26/30, Loss: 0.7238296390503876
Epoch 27/30, Loss: 0.723602531

In [ ]:
# Save the trained model
torch.save(model.state_dict(), './Workspace/EEG_caption/model_weights/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")

Model saved as PixelProjector.bin
